# Parkinson's Machine Learning Classification

## Table of Contents
1. [Initial Data Analysis](#Initial-Data-Analysis)
1. [Initial Model Benchmarks](#Initial-Model-Benchmarks)

## Initial Data Analysis 
The data set obviously has a class imbalance (Control 32%, Parkinsons 53%, MSA 7%, PSP 7%).  We will need to resample the data or use an algorithm like Decision Trees that are not influenced as heavily by class imbalance.  We should also take care to see how well the under-represented classes are being predicted when evaluating accuracy of our models.

Feature appear relatively normally-distributed between groups.  Nothing particularly interesting jumping out by looking at covariance and pearson correlations

[See Data Exploration Notebook for more details](Data Exploration/Data_Exploration.ipynb)

## Initial Model Benchmarks
8 initial models were tested for __multiclass classification__ (I would ideally like to avoid composing results of multiple binary classifiers since the cost of that composition is great):
* Logistic Regression
* SVC with Linear Kernel
* SVC with RBF Kernel
* k-Nearest Neighbors
* Decision Trees
* Artificial Neural Networks
* Naive Bayes
* AdaBoost

These models were run with 5-fold (stratified) cross validation on 80% of the training data (20% kept for holdout validation).  To address the problem of class imbalance, the data was resampled (upsampling of the minority classes) to ensure that the minority classes were well represented in the models.  Alternative imbalanced class corrections can be explored in the future.   No additional preprocessing of the data was performed (scaling, feature selceti)

For most models default parameters were used (with minor tweaking) 

[See Modeling Benchmark Notebook for more details](Model Benchmarks/ModelingBenchmarks.ipynb)

### Test Subsection